# Test our Ecohydrology module

This notebook is to test that importing and using our implementation of the Ecohydrology model works. 

In [1]:
from ecohydr_mod import EcoHyd
import numpy as np

In [2]:
WSA_array = np.zeros((101,101))

Ecohyd_model = EcoHyd()

In [3]:
Ecohyd_model.stepper(WSA_array)

soil moisture sat.: [ 0.61138623  0.61120936  0.61103477 ...,  0.66400311  0.66403955
  0.66407648]
live biomass:  [ 0.  0.  0. ...,  0.  0.  0.]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
soil moisture sat.: [ 0.53888644  0.53858039  0.53827833 ...,  0.62655365  0.62661027
  0.62666762]
live biomass:  [ 0.  0.  0. ...,  0.  0.  0.]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
soil moisture sat.: [ 0.4804794   0.4800737   0.47967358 ...,  0.60402954  0.60410696
  0.60418538]
live biomass:  [ 0.  0.  0. ...,  0.  0.  0.]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
ET:  [ 13.36071899  13.39290062  13.42465873 ...,   3.19250858   3.1849562
   3.17730311]
soil moisture sat.: [ 

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [4]:
import matplotlib.pyplot as plt

In [8]:
Ecohyd_model.mg.at_cell['vegetation__water_stress']

array([ 0.87102151,  0.87117969,  0.87133537, ...,  0.75530801,
        0.75507928,  0.75484684])

In [ ]:
Ecohyd_model.mg.at_cell['']